# LSTM Model

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from dataclasses import dataclass
from typing import Union, Tuple, List

In [ ]:
# Read the data.
df = pd.read_csv('smoothed_data_kansas.csv')
x = np.concatenate((df['x_19'], df['x_20'], df['x_22']), axis=0)
# NDVI time series.
y = np.concatenate((df['y_19'], df['y_20'], df['y_22']), axis=0)
# Generate the time vector in days after planting.
time = [i for i in range(len(y))]

In [ ]:
plt.title('NDVI time series for Kansas')
plt.plot(time, y, 'b', label='2019-2022')
plt.legend()
plt.show()

In [ ]:
# Time series parameters.
@dataclass
class PARAMETERS:
  TIME, SERIES = time, y
  # Number of days to be used on the training set.
  SPLIT_TIME = 200
  # Data points to make our prediction.
  WINDOW_SIZE = 25
  # How many items will we supply per batch.
  BATCH_SIZE = 32
  # Define the Tensorflow sample buffer.
  SHUFFLE_BUFFER_SIZE = 1000
  # Number of epochs.
  EPOCHS = 50
  # Next t steps to predeict.
  PREDICT_STEPS = 150

In [ ]:
def train_val_split(
  time: Union[List, np.ndarray],
  series: Union[List, np.ndarray],
  time_step: int = PARAMETERS.SPLIT_TIME
) -> Tuple[np.ndarray, ...]:
  """
  Divide the time series into training and validation set.
  """
  assert isinstance(time, (np.ndarray, list))
  
  # Training time vector.
  time_train = time[:time_step]
  # NDVI series vector.
  series_train = series[:time_step]
  # Validation time vector.
  time_valid = time[time_step:]
  # NDVI series vector.
  series_valid = series[time_step:]
  
  return time_train, series_train, time_valid, series_valid

def windowed_dataset(
	series: Union[List, np.ndarray],
	window_size: int = PARAMETERS.WINDOW_SIZE,
	batch_size: int = PARAMETERS.BATCH_SIZE,
	shuffle_buffer: int = PARAMETERS.SHUFFLE_BUFFER_SIZE
) -> tf.data.Dataset:
	"""
	We create time windows to create X and y features.
	For example, if we choose a window of 30, we will create a dataset formed by 30 points as X
	"""
	# Create a TensorFlow dataset from np.arrays or lists.
	dataset = tf.data.Dataset.from_tensor_slices(series)
	# Create a DataFrame of windows.
	dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
	# Flat the vectors and make batches of the data.
	dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
	# Randomly shuffle the data.
	dataset = dataset.shuffle(shuffle_buffer)
	# Transform the data to the format X, y.
	dataset = dataset.map(lambda window: (window[:-1], window[-1]))
	# Prefetch.
	dataset = dataset.batch(batch_size).prefetch(1)
	return dataset

In [ ]:
# Split the data into training and validation set
time_train, series_train, time_valid, series_valid = train_val_split(PARAMETERS.TIME, PARAMETERS.SERIES)

In [ ]:
# Create the dataset with time windows.
dataset = windowed_dataset(series_train)

In [ ]:
class EarlyStopping(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('mae') < 0.03):
      print("\nMAEthreshold reached. Training stopped.")
      self.model.stop_training = True

# Let's create an object of our class and assign it to a variable
early_stopping = EarlyStopping()

In [ ]:
def create_model():
  """
  Function to define a sequential model.
  Appropriate for a plain stack of layers where 
  each layer has exactly ONE input tensor and ONE output tensor.
  """
  # Set a random seed.
  tf.random.set_seed(51)

  # Uncompiled model.
  model = tf.keras.models.Sequential([
      # Lambda layer to handle the input data format. 
      tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), 
                              input_shape=[None]),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1024, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
      tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
      # Layer of prediction.
      tf.keras.layers.Dense(1),
  ])
  
  # Compile model for training.
  model.compile(
    loss=tf.keras.losses.Huber(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics=['mae']
  )

  return model

In [ ]:
# Call and create the model.
model = create_model()

In [ ]:
# Training and assign the callback
history = model.fit(dataset, epochs=PARAMETERS.EPOCHS, callbacks=[early_stopping])

In [ ]:
def compute_metrics(
  true_series : Union[List, np.ndarray],
  forecast : Union[List, np.ndarray]
) -> Tuple[float, float]:
  """
  Print MSE and MAE
  """
  mse = tf.keras.metrics.mean_squared_error(true_series, forecast).numpy()
  mae = tf.keras.metrics.mean_absolute_error(true_series, forecast).numpy()

  return mse, mae

In [ ]:
def model_forecast(
  model, 
  series: np.ndarray, 
  window_size: int
) -> np.ndarray:
  """
  Converts the input series into a dataset with time windows for forecasting.
  """
  ds = tf.data.Dataset.from_tensor_slices(series)
  ds = ds.window(window_size, shift=1, drop_remainder=True)
  ds = ds.flat_map(lambda w: w.batch(window_size))
  ds = ds.batch(32).prefetch(1)
  forecast = model.predict(ds)
  
  return forecast

In [ ]:
# Prediction on the whole series.
all_forecast = model_forecast(model, PARAMETERS.SERIES, PARAMETERS.WINDOW_SIZE).squeeze()

# Validation portion.
val_forecast = all_forecast[PARAMETERS.SPLIT_TIME - PARAMETERS.WINDOW_SIZE:-1]

In [ ]:
plt.figure(figsize=(10, 6))
plt.plot(series_valid, label="Validation set")
plt.plot(val_forecast, label="Predicted NDVI")
plt.xlabel("Days after planting")
plt.ylabel("NDVI")
plt.legend()
plt.show()

In [ ]:
mse, mae = compute_metrics(series_valid, val_forecast)
print(f"mse: {mse:5f}, mae: {mae:5f}")

In [ ]:
# Get the forecast of the model.
new_forecast = []

for i in range(PARAMETERS.PREDICT_STEPS):
  new_forecast_series = PARAMETERS.SERIES[-PARAMETERS.WINDOW_SIZE:]  
  pred = model.predict(new_forecast_series[np.newaxis])
  new_forecast.append(pred.reshape(-1))
  PARAMETERS.SERIES = np.append(PARAMETERS.SERIES, pred)

# Crete the new time vector.
new_time = [i for i in range(len(PARAMETERS.TIME) + 1, len(PARAMETERS.TIME) + PARAMETERS.PREDICT_STEPS + 1)]

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(PARAMETERS.TIME[:300], PARAMETERS.SERIES[:300], label="last 400 points of time series")
plt.plot(new_time, new_forecast, color="red", label="prediction")
plt.legend()
plt.show()